In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/btp-data/train.csv


In [2]:
import logging
logging.getLogger('transformers').setLevel(logging.ERROR)


In [3]:
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.8 MB/s eta 0:00:00


In [4]:
df = pd.read_csv("/kaggle/input/btp-data/train.csv")

In [5]:
df['entity_name'].unique()


array(['item_weight', 'item_volume', 'voltage', 'wattage',
       'maximum_weight_recommendation', 'height', 'depth', 'width'],
      dtype=object)

In [6]:
import torch

In [7]:
import requests
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM 

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32


# Load the Florence-2 model and processor
model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large-ft", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)


config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
import requests
from io import BytesIO
from PIL import Image, UnidentifiedImageError

def extract_text_from_image(image_source):
    try:

        if image_source.startswith("http://") or image_source.startswith("https://"):
            response = requests.get(image_source)
            image = Image.open(BytesIO(response.content))
        else:
            image = Image.open(image_source)
        
        prompt = "<OCR>"
        inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

        # Generate output
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=524,
            num_beams=3,
            do_sample=False
        )

        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        parsed_answer = processor.post_process_generation(generated_text, task="<OCR>", image_size=(image.width, image.height))

        return parsed_answer

    except UnidentifiedImageError:
        return "Error loading image"
    except requests.exceptions.RequestException:
        return "Error retrieving image from URL"
    except Exception as e:
        return f"Unexpected error: {e}"
        

In [9]:
import time
st = time.time()
# Example usage
image_path = "https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg" 
text_data = extract_text_from_image(image_path)
print("Extracted Text:", text_data)
end = time.time()
print(end - st)

Extracted Text: {'<OCR>': "PROPÉS'NATUREINGREDIENT MÉAGERMULTI-USAGETERRE DESOMMIÈRES100% NATURLAngue 100% paré à naturelle, l'erre deau parfumément pour un projetifrice de l'eau. La naturel de la naturel, des légumesparfumeur, blanc et sautifrice moquettesapples, gras et sauteur moquentesdeodoriser le linge.télégéant, bertoneeOngage confortable, Séden l'égé.Pour les couleurs de léger, de la couleur de la création de la bouleurPour l'amour de la vie en fronction de la fronciet100%Lait PRODUCE NOS2 500 gLait Progérationnelle de la méditeur d'un mégèneur.2 017070 78741"}
4.597406625747681


In [14]:
df1 = df[255000 : 260000]

In [15]:
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import time
import os

def safe_extract_text(url):
    try:
        return extract_text_from_image(url)
    except Exception as e:
        print(f"Error with URL {url}: {e}")
        return None

def process_chunk(chunk):
    tqdm.pandas()
    chunk['extracted_text'] = chunk['image_link'].progress_apply(safe_extract_text)
    return chunk

def process_dataset_images(dataset, chunk_size=1000, num_workers=4, save_file="processed_data_255000_260000.csv"):

    if os.path.exists(save_file):
        processed_data = pd.read_csv(save_file)
        start_idx = len(processed_data)
        dataset = dataset[start_idx:]
    else:
        processed_data = pd.DataFrame()
        start_idx = 0
    
    for i in range(start_idx, len(dataset), chunk_size):
        chunk = dataset[i:i+chunk_size]
        

        with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
            processed_chunks = list(tqdm(executor.map(process_chunk, [chunk]), total=len(chunk)))
        

        processed_data = pd.concat([processed_data, pd.concat(processed_chunks)])
        processed_data.to_csv(save_file, index=False)
        print(f"Saved progress at row {i + chunk_size}")
    
    return processed_data

processed_dataset = process_dataset_images(df1, chunk_size=1000, num_workers=4, save_file="processed_data_255000_260000.csv")
df_test = pd.read_csv("processed_data_255000_260000.csv")
df_test.head()



100%|██████████| 1000/1000 [16:18<00:00,  1.02it/s]
/tmp/ipykernel_30/1990432721.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['extracted_text'] = chunk['image_link'].progress_apply(safe_extract_text)

  0%|          | 1/1000 [16:18<271:38:04, 978.86s/it]


Saved progress at row 1000


100%|██████████| 1000/1000 [16:16<00:00,  1.02it/s]
/tmp/ipykernel_30/1990432721.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['extracted_text'] = chunk['image_link'].progress_apply(safe_extract_text)

  0%|          | 1/1000 [16:16<270:56:03, 976.34s/it]


Saved progress at row 2000



100%|██████████| 1000/1000 [16:04<00:00,  1.04it/s]
/tmp/ipykernel_30/1990432721.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['extracted_text'] = chunk['image_link'].progress_apply(safe_extract_text)
  0%|          | 1/1000 [16:04<267:39:17, 964.52s/it]


...

100%|██████████| 1000/1000 [16:34<00:00,  1.01it/s]
/tmp/ipykernel_30/1990432721.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['extracted_text'] = chunk['image_link'].progress_apply(safe_extract_text)

  0%|          | 1/1000 [16:34<276:04:24, 994.86s/it]


Saved progress at row 4000


100%|██████████| 1000/1000 [16:20<00:00,  1.02it/s]
/tmp/ipykernel_30/1990432721.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['extracted_text'] = chunk['image_link'].progress_apply(safe_extract_text)
  0%|          | 1/1000 [16:20<272:04:56, 980.48s/it]

Saved progress at row 5000


,image_link,group_id,entity_name,entity_value,extracted_text
0,https://m.media-amazon.com/images/I/41pRlBORqa...,279199,height,50.0 centimetre,{'<OCR>': '50cm23cm32.5cm'}
1,https://m.media-amazon.com/images/I/6180sWRsWk...,245959,height,38.0 centimetre,{'<OCR>': 'Product Size38cm/14.96inch28cm/11in...
2,https://m.media-amazon.com/images/I/51PUadKGaF...,810266,height,6.7 inch,{'<OCR>': '12OZ3.3in6.7in3.7inch'}
3,https://m.media-amazon.com/images/I/51k+jFAcRX...,187993,height,23.5 centimetre,{'<OCR>': 'CoverFilter23.5cm/g.1inchFood Grade...
4,https://m.media-amazon.com/images/I/51ubvUbtnW...,945489,height,20.2 centimetre,{'<OCR>': 'Capacity: 250ml20.2cm5.3cm'}


In [ ]:
import time
time.sleep(10000)

In [ ]:
import time
time.sleep(10000)